In [1]:
import pandas as pd
import sqlite3

In [2]:
excel_file = "/Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/Dataset/a_IBM Telco Customers Churn Datasets.xlsx"
database =  "telco_churn.db"

In [3]:
excel_data = pd.ExcelFile(excel_file)
all_sheets = excel_data.sheet_names


# exclude the Population and Telco_churn sheets
sheets_included = all_sheets[:-2]

# create a connection
connect = sqlite3.connect(database)

# load each sheet into the database
for sheet in sheets_included:
    df = excel_data.parse(sheet)
    #convert the data to tables to use in sql
    df.to_sql(sheet, connect, if_exists = 'replace', index =  'False')
    print(f"Loaded sheet: {sheet} into the database")

connect.close()
print("Sucessfully loaded all sheets")

Loaded sheet: Customer_Info into the database
Loaded sheet: Service_Options into the database
Loaded sheet: Online_Services into the database
Loaded sheet: Payment_Info into the database
Loaded sheet: Status_Analysis into the database
Loaded sheet: Location_Data into the database
Sucessfully loaded all sheets


In [4]:
%load_ext sql

In [5]:
%sql sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db

### Churn Rate

In [7]:
%%sql
SELECT 
    customer_status AS 'Customer Status', 
    COUNT(customer_id) AS 'Number of Customers',
    ROUND((COUNT(customer_id)*100.0)/(SELECT COUNT(customer_id) FROM Status_Analysis), 1) AS 'Percentage of customers'
FROM Status_Analysis
GROUP BY customer_status

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


Customer Status,Number of Customers,Percentage of customers
Churned,1869,26.5
Joined,454,6.4
Stayed,4720,67.0


<b>Churn rate: 26.5% </b> </br>
Churned customers account for over a quarter of total number of customers, which is considered high. This means that we should look at the reasons or similar patterns why many customers left Telco. </br>
</br>
<b>Percent of new customers: 6.4%</b> <br>
This indicates that while there are still new customers joining, the rate of joining is much lower than the rate of customers leaving. Telco needs to improve their strategies to attract new customers.</br>
</br>
<b>Percent of loyal customers: 67%</b> <br>
Large amount of customers choose to stay with Telco. This shows the need for Telco to improve customer satisfaction while working on loyalty programs for existing customers.

### Churn Rate by Gender, Age Group, and Number of dependents

In [51]:
%%sql 
SELECT c.gender, COUNT(c.customer_id)
FROM Customer_Info as c
JOIN Status_Analysis as s
ON c.customer_id = s.customer_id
WHERE customer_status = 'Churned'
GROUP BY c.gender

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


gender,COUNT(c.customer_id)
Female,939
Male,930


<b> Impact of gender on churn rate </b> </br> 
Number of customers who left is equal for both male and female. There is no evidence that genders affect customers' decision to leave in this dataset.

In [11]:
%%sql 
SELECT 
    CASE 
        WHEN customers.age BETWEEN AND THEN
        
    END AS 'Age group'
FROM Customer_Info as c
JOIN Status_Analysis as s
USING (customer_id) 

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
(sqlite3.OperationalError) near "AND": syntax error
[SQL: SELECT 
    CASE 
        WHEN customers.age BETWEEN AND THEN
        
    END AS 'Age group'
FROM Customer_Info as c
JOIN Status_Analysis as s
USING (customer_id)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


<b> Impact of age on churn rate </b> </br> 


##### Relationship between churn status and number of dependents

### % of customers for each type of payment, by customer status

In [39]:
%%sql
-- This is onot helpful 
SELECT 
    s.customer_status, 
    p.payment_method, 
    COUNT(p.customer_id) AS 'Number of customers',
    SUM(COUNT(p.customer_id)) OVER (PARTITION BY p.payment_method) AS 'Total customers by payment method',
    ROUND(COUNT(p.customer_id)*100.0/SUM(COUNT(p.customer_id)) OVER (PARTITION BY p.payment_method), 1) AS 'Percentage by payment method (%)'
FROM Payment_Info AS p 
JOIN Status_Analysis AS s
USING (customer_id)
GROUP BY p.payment_method, s.customer_status

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


customer_status,payment_method,Number of customers,Total customers by payment method,Percentage by payment method (%)
Churned,Bank transfer (automatic),258,1544,16.7
Joined,Bank transfer (automatic),33,1544,2.1
Stayed,Bank transfer (automatic),1253,1544,81.2
Churned,Credit card (automatic),232,1522,15.2
Joined,Credit card (automatic),39,1522,2.6
Stayed,Credit card (automatic),1251,1522,82.2
Churned,Electronic check,1071,2365,45.3
Joined,Electronic check,148,2365,6.3
Stayed,Electronic check,1146,2365,48.5
Churned,Mailed check,308,1612,19.1


In [37]:
%%sql
SELECT 
    s.customer_status, 
    p.payment_method, 
    COUNT(p.customer_id) AS 'Number of customers',
    SUM(COUNT(s.customer_id)) OVER (PARTITION BY s.customer_status) AS 'Total customers by status',
    ROUND(COUNT(p.customer_id)*100.0/SUM(COUNT(s.customer_id)) OVER (PARTITION BY s.customer_status), 1) AS 'Percentage by customer status (%)'
FROM Payment_Info AS p 
JOIN Status_Analysis AS s
USING (customer_id)
GROUP BY p.payment_method, s.customer_status

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


customer_status,payment_method,Number of customers,Total customers by status,Percentage by customer status (%)
Churned,Bank transfer (automatic),258,1869,13.8
Churned,Credit card (automatic),232,1869,12.4
Churned,Electronic check,1071,1869,57.3
Churned,Mailed check,308,1869,16.5
Joined,Bank transfer (automatic),33,454,7.3
Joined,Credit card (automatic),39,454,8.6
Joined,Electronic check,148,454,32.6
Joined,Mailed check,234,454,51.5
Stayed,Bank transfer (automatic),1253,4720,26.5
Stayed,Credit card (automatic),1251,4720,26.5


Highest churn rate occurs in customers who use electronic check. Customers who stay with Telco use all types of payment methods equally. 

## Analysis of current offers, plans, and strategies of Telco and how they impact churn decisions 

### Main churn reason

In [84]:
%%sql 
SELECT 
    churn_category AS 'Reason', 
    COUNT(customer_id) AS 'Number of customers'
FROM Status_Analysis
WHERE customer_status = 'Churned'
GROUP BY churn_category
ORDER BY COUNT(customer_id) DESC

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


Reason,Number of customers
Competitor,841
Attitude,314
Dissatisfaction,303
Price,211
Other,200


<b> Main reason for churning: Competitor </b> </br> 
Significant number of customers (over 800 customers) decide to leave Telco for other companies. This means that there is a chance that offers or plans by Telco is not as appealing as other competitors. Since we don't have information on other companies' offerings, we can use our current dataset to narrow down factors that can lead to higher churn rate of customers. 


### Examination of current offers and their responding churn rate

In [106]:
%%sql 
SELECT offer, COUNT(customer_id) AS 'Number of customers who churned'
FROM Service_Options
JOIN Status_Analysis
USING (customer_id)
WHERE customer_status = 'Churned'
GROUP BY offer
ORDER BY COUNT(customer_id) DESC

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


offer,Number of customers who churned
None,1051
Offer E,426
Offer D,161
Offer B,101
Offer C,95
Offer A,35


Customers who received no offer tend to leave Telco more than those that receive certain type of offer. Offer A seems to be the best in terms of keeping loyal customers. 

<b> Likeliness to refer to a friend based on the offer they received </b>

In [237]:
%%sql
SELECT 
    offer, 
    COUNT(customer_id) AS 'Number of customers who referred a friend'
FROM Service_Options
WHERE referred_a_friend = 'Yes'
GROUP BY offer

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


offer,Number of customers who referred a friend
None,1758
Offer A,388
Offer B,490
Offer C,184
Offer D,220
Offer E,182


While customers with none offer have a higher tendency to churn, they are surprisingly more likely to refer to a friend.

#### Do customers with higher tenure more likely to refer to more friends?

In [180]:
%%sql 
SELECT 
    CASE
        WHEN tenure BETWEEN 0 AND 6 THEN 'High risk of churn'
        WHEN tenure BETWEEN 7 AND 12 THEN 'Early tenure'
        WHEN tenure BETWEEN 13 AND 24 THEN 'Moderate tenure'
        WHEN tenure BETWEEN 25 AND 36 THEN 'Satisfied customer'
        WHEN tenure BETWEEN 37 AND 60 THEN 'Long-term customer'
        WHEN tenure>60 THEN 'Long-term customer'
    END AS loyalty_status, 
    COUNT(customer_id) AS 'Number of customers who referred a friend'
FROM Service_Options
WHERE referred_a_friend = 'Yes'
GROUP BY loyalty_status
ORDER BY COUNT(customer_id) DESC

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


loyalty_status,Number of customers
Long-term customer,1895
Moderate tenure,420
Satisfied customer,381
High risk of churn,300
Early tenure,226


In [205]:
%%sql 
SELECT 
    CASE
        WHEN tenure BETWEEN 0 AND 6 THEN 'High risk of churn'
        WHEN tenure BETWEEN 7 AND 12 THEN 'Early tenure'
        WHEN tenure BETWEEN 13 AND 24 THEN 'Moderate tenure'
        WHEN tenure BETWEEN 25 AND 36 THEN 'Satisfied customer'
        WHEN tenure BETWEEN 37 AND 60 THEN 'Long-term customer'
        WHEN tenure>60 THEN 'Long-term customer'
    END AS loyalty_status, 
    COUNT(customer_id) AS 'Number of customers who churned'
FROM Service_Options
JOIN Status_Analysis
USING (customer_id)
WHERE customer_status = 'Churned'
GROUP BY loyalty_status
ORDER BY COUNT(customer_id) DESC

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


loyalty_status,Number of customers who churned
High risk of churn,784
Long-term customer,358
Moderate tenure,294
Early tenure,253
Satisfied customer,180


In [235]:
%%sql 
SELECT 
    CASE
        WHEN tenure BETWEEN 0 AND 6 THEN 'High risk of churn'
        WHEN tenure BETWEEN 7 AND 12 THEN 'Early tenure'
        WHEN tenure BETWEEN 13 AND 24 THEN 'Moderate tenure'
        WHEN tenure BETWEEN 25 AND 36 THEN 'Satisfied customer'
        WHEN tenure BETWEEN 37 AND 60 THEN 'Long-term customer'
        WHEN tenure>60 THEN 'Long-term customer'
    END AS loyalty_status, 
    COUNT(CASE WHEN customer_status = 'Churned' THEN customer_id END) AS churn_customers,
    COUNT(CASE WHEN referred_a_friend = 'Yes' THEN customer_id END) AS referred_friends
FROM Service_Options
JOIN Status_Analysis
USING (customer_id)
GROUP BY loyalty_status
ORDER BY referred_friends DESC

 * sqlite:////Users/tu/Documents/Tu/Data_Portfolio/Customer_churn/telco_churn.db
Done.


loyalty_status,churn_customers,referred_friends
Long-term customer,358,1895
Moderate tenure,294,420
Satisfied customer,180,381
High risk of churn,784,300
Early tenure,253,226
